# **Libraries**

In [65]:
import pandas as pd

import nltk
nltk.download("all", quiet=True)
from nltk.corpus import stopwords
from nltk import word_tokenize
import string #puntuacion
from nltk.stem import WordNetLemmatizer #Lematizador

# **Reading Data**

In [66]:
raw_data = pd.read_csv("../data/raw_data.csv")
raw_data.head()

,element_id,site_id,seller_id,Dominio_normalizado,Titulo,Marca,Precio,Rule,Score,Total,Moderado,Fake,Rollback,FK_TEST1,FK_TEST2
0,ARG1000130348,ARGENTINA,924394736,COMPUTER_PROCESSORS,Microprocesador Intel Core I3 12100 12mb Bx807...,Intel,222599.0,NaN,0.11,1,0,0,0,0,0
1,ARG1002056504,ARGENTINA,60385780,COMPUTER_MONITORS,Monitor Gamer Samsung Con Pantalla De 25 60hz...,Samsung,767000.0,NaN,0.29,1,0,0,0,0,0
2,ARG1002630435,ARGENTINA,1085316688,SUNGLASSES,Anteojos De Sol Polarizados Ray-ban Erika Clas...,Ray-Ban,246510.0,NaN,0.51,1,0,0,0,0,0
3,ARG1003499360,ARGENTINA,189266308,HARD_DRIVES_AND_SSDS,Ssd Externo Kingston Xs1000 1tb Negro Usb 3.2 ...,Kingston,179999.0,NaN,0.32,1,0,0,0,0,0
4,ARG1004623658,ARGENTINA,127503700,RAM_MEMORY_MODULES,Memoria Ram Valueram Color Verde 8gb 1 Kingsto...,Kingston,22399.0,NaN,0.80,1,0,0,0,0,0


# **Quick Data Understanding**

Data Types

In [67]:
raw_data.dtypes

element_id              object
site_id                 object
seller_id                int64
Dominio_normalizado     object
Titulo                  object
Marca                   object
Precio                 float64
Rule                    object
Score                  float64
Total                    int64
Moderado                 int64
Fake                     int64
Rollback                 int64
FK_TEST1                 int64
FK_TEST2                 int64
dtype: object

Duplicated Vaues

In [68]:
raw_data.duplicated().sum()

0

Nulls

In [69]:
raw_data.isna().sum()
pd.DataFrame(raw_data.isna().sum() / len(raw_data) * 100).sort_values(0,ascending=False)

,0
Rule,89.403547
element_id,0.000000
site_id,0.000000
seller_id,0.000000
Dominio_normalizado,0.000000
Titulo,0.000000
Marca,0.000000
Precio,0.000000
Score,0.000000
Total,0.000000


Review Some Distributions

In [70]:
raw_data.describe()

,seller_id,Precio,Score,Total,Moderado,Fake,Rollback,FK_TEST1,FK_TEST2
count,4.037200e+04,4.037200e+04,40372.000000,40372.0,40372.000000,40372.000000,40372.000000,40372.000000,40372.000000
mean,1.911603e+09,2.455866e+05,0.428654,1.0,0.105965,0.111191,0.007580,0.047483,0.007282
std,1.784332e+09,6.435465e+05,0.330362,0.0,0.307796,0.314372,0.086731,0.212673,0.085026
min,4.281520e+05,1.117000e+01,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.440346e+08,6.767050e+02,0.150000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.217641e+09,4.299000e+03,0.310000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.609587e+09,1.549900e+05,0.810000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
max,8.880437e+09,1.289990e+07,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000


# **Building Final Dataset**

In [71]:
process_data = raw_data.copy()
process_data["Rule"] = process_data["Rule"].fillna("NOT_MODERATED")
process_data

,element_id,site_id,seller_id,Dominio_normalizado,Titulo,Marca,Precio,Rule,Score,Total,Moderado,Fake,Rollback,FK_TEST1,FK_TEST2
0,ARG1000130348,ARGENTINA,924394736,COMPUTER_PROCESSORS,Microprocesador Intel Core I3 12100 12mb Bx807...,Intel,222599.0,NOT_MODERATED,0.11,1,0,0,0,0,0
1,ARG1002056504,ARGENTINA,60385780,COMPUTER_MONITORS,Monitor Gamer Samsung Con Pantalla De 25 60hz...,Samsung,767000.0,NOT_MODERATED,0.29,1,0,0,0,0,0
2,ARG1002630435,ARGENTINA,1085316688,SUNGLASSES,Anteojos De Sol Polarizados Ray-ban Erika Clas...,Ray-Ban,246510.0,NOT_MODERATED,0.51,1,0,0,0,0,0
3,ARG1003499360,ARGENTINA,189266308,HARD_DRIVES_AND_SSDS,Ssd Externo Kingston Xs1000 1tb Negro Usb 3.2 ...,Kingston,179999.0,NOT_MODERATED,0.32,1,0,0,0,0,0
4,ARG1004623658,ARGENTINA,127503700,RAM_MEMORY_MODULES,Memoria Ram Valueram Color Verde 8gb 1 Kingsto...,Kingston,22399.0,NOT_MODERATED,0.80,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40367,PER9979201495,PERU,1906413812,SPEAKERS,Bocina Jbl Charge 5 Jblcharge5 Portátil Con Bl...,JBL,799.0,NOT_MODERATED,0.13,1,0,0,0,0,0
40368,PER9979619481,PERU,699068980,HEADPHONES,Audifonos Jbl Tune 520 Bt Bluetooth On Ear Col...,JBL,299.0,NOT_MODERATED,0.80,1,0,0,0,0,0
40369,PER9980515950,PERU,1510316564,SMARTWATCHES,Apple Watch Series 10 Gps Caja De Aluminio Ne...,Apple,2499.0,NOT_MODERATED,0.83,1,0,0,0,0,0
40370,PER999421010,PERU,2495083952,WRISTWATCHES,Reloj Casio W218wd Acero Inoxidable Alarma Sum...,Casio,389.0,NOT_MODERATED,0.10,1,0,0,0,0,0


In [72]:
process_data.to_csv("../data/processed_data.csv", index=False)

# **Building Text Dataset**

In [73]:
len(process_data[process_data["site_id"] != "BRASIL"]["Titulo"])

26230

In [74]:
len(process_data[process_data["site_id"] == "BRASIL"]["Titulo"])

14142

Normalizing Data

In [75]:
def preprocess(text, language: str):
    stop_words = set(stopwords.words(language))
    wordnet_lemmatizer = WordNetLemmatizer() 
    text = text.lower() 
    text = ''.join([word for word in text if word not in string.punctuation]) 
    tokens = word_tokenize(text) 
    tokens = [word for word in tokens if word not in stop_words] 
    lemas = [wordnet_lemmatizer.lemmatize(word) for word in tokens] 
    return ' '.join(lemas), len(lemas)

text_brasil= process_data[process_data["site_id"] == "BRASIL"]["Titulo"].apply(lambda x: preprocess(x, "portuguese")).apply(pd.Series).rename(columns={0: "normalized", 1: "number_of_words"})
text_spanish = process_data[process_data["site_id"] != "BRASIL"]["Titulo"].apply(lambda x: preprocess(x, "spanish")).apply(pd.Series).rename(columns={0: "normalized", 1: "number_of_words"})
text_all_countries = pd.concat([text_spanish, text_brasil], axis=0).sort_index()
df_raw_text = pd.concat([process_data, text_all_countries], axis=1)

In [76]:
df_raw_text.head()

,element_id,site_id,seller_id,Dominio_normalizado,Titulo,Marca,Precio,Rule,Score,Total,Moderado,Fake,Rollback,FK_TEST1,FK_TEST2,normalized,number_of_words
0,ARG1000130348,ARGENTINA,924394736,COMPUTER_PROCESSORS,Microprocesador Intel Core I3 12100 12mb Bx807...,Intel,222599.0,NOT_MODERATED,0.11,1,0,0,0,0,0,microprocesador intel core i3 12100 12mb bx807...,23
1,ARG1002056504,ARGENTINA,60385780,COMPUTER_MONITORS,Monitor Gamer Samsung Con Pantalla De 25 60hz...,Samsung,767000.0,NOT_MODERATED,0.29,1,0,0,0,0,0,monitor gamer samsung pantalla 25 60hz 220v re...,10
2,ARG1002630435,ARGENTINA,1085316688,SUNGLASSES,Anteojos De Sol Polarizados Ray-ban Erika Clas...,Ray-Ban,246510.0,NOT_MODERATED,0.51,1,0,0,0,0,0,anteojos sol polarizados rayban erika classic ...,13
3,ARG1003499360,ARGENTINA,189266308,HARD_DRIVES_AND_SSDS,Ssd Externo Kingston Xs1000 1tb Negro Usb 3.2 ...,Kingston,179999.0,NOT_MODERATED,0.32,1,0,0,0,0,0,ssd externo kingston xs1000 1tb negro usb 32 g...,10
4,ARG1004623658,ARGENTINA,127503700,RAM_MEMORY_MODULES,Memoria Ram Valueram Color Verde 8gb 1 Kingsto...,Kingston,22399.0,NOT_MODERATED,0.80,1,0,0,0,0,0,memoria ram valueram color verde 8gb 1 kingsto...,9


## _Análisis por País_

In [77]:
df_pais = df_raw_text.groupby(["site_id"]).agg({"Titulo": lambda x: " ".join(x), "normalized": lambda x: " ".join(x), "seller_id": "count", "Dominio_normalizado": "count", "Titulo": "count", "Marca": "count", "Precio": "mean", "number_of_words": [("mean", "mean"), ("min", "min"), ("max", "max")]}).reset_index()
df_pais.columns = ['_'.join(col).strip() for col in df_pais.columns.values]
# df_pais = df_pais.rename(columns={"Genero_": "genero", "Cancion_<lambda>": "cancion", "normalize_<lambda>": "normalize", "Artista_count": "n_canciones"})
df_pais

,site_id_,Titulo_count,normalized_<lambda>,seller_id_count,Dominio_normalizado_count,Marca_count,Precio_mean,number_of_words_mean,number_of_words_min,number_of_words_max
0,ARGENTINA,11938,microprocesador intel core i3 12100 12mb bx807...,11938,11938,11938,609388.477741,12.256576,3,39
1,BRASIL,14142,cartão memória kingston sds264gb canvas select...,14142,14142,14142,1348.618944,9.832273,3,26
2,CHILE,2116,tóner hp 206a color cian tarjeta memoria cfexp...,2116,2116,2116,260282.366257,8.956049,2,25
3,COLOMBIA,2044,samsung galaxy a05s dual sim 128 gb plateado 4...,2044,2044,2044,988805.057241,10.043053,2,30
4,MEXICO,8378,lentes sol carolina herrera mujer her 0177s 20...,8378,8378,8378,5636.939949,10.055980,2,35
5,PERU,1754,apple airpods max morado distribuidor autoriza...,1754,1754,1754,1009.432691,9.127708,3,34
